In [49]:
import pandas as pd
import audio2numpy as a2n
import scipy
import transformers
import torch
import random
import IPython

In [46]:
class DataGenerator:

    def __init__(self):
        self.df = pd.read_csv(
            'data/common-voice-corpus-12.0-delta-2022-12-07/en/validated.tsv', 
            sep= '\t'
        )

    def getDatapointAt(self,i):
        row      = self.df.iloc[i]
        sentence = row['sentence']
        path     = row['path'    ]
        path     = f'data/common-voice-corpus-12.0-delta-2022-12-07/en/clips/{path}'
        sig, sr  = a2n.audio_from_file(path)
        sig      = scipy.signal.resample(sig, num= len(sig) * 16_000 // sr)
        
        return path, sig, sentence
    
    def getRandomDataPoint(self):
        i = random.randint(0,self.df.shape[0])
        return self.getDatapointAt(i)
    
    def getGenerator(self):
        for i in range(self.df.shape[0]):
            yield self.getDatapointAt(i)


In [57]:
modelId = 'facebook/wav2vec2-large-960h-lv60-self'

model     = transformers.Wav2Vec2ForCTC   .from_pretrained(modelId)
processor = transformers.Wav2Vec2Processor.from_pretrained(modelId)

dataGenerator = DataGenerator()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
path, audio, sentence = dataGenerator.getRandomDataPoint()

audio      = processor(audio, return_tensors= 'pt', padding= 'longest').input_values
prediction = model(audio).logits
prediction = torch.argmax(prediction, dim=-1)
prediction = processor.batch_decode(prediction)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [75]:
print(f'''
    prediction      : {prediction}
    orignal sentence: {sentence}
    
    audio:
''')
IPython.display.Audio(path)


    prediction      : THERE IS NO RECORD OF A COMPANY SLOOP LOSING ITS BOAT
    orignal sentence: There is no record of a Company sloop losing its boat.
    
    audio:

